# Defining, compiling, and running stencils

This notebook demonstrates how stencils can be defined, compiled and executed.

## Configuration

First, let's define the configuration such as the size of the computational domain as well as the number of halo points and the GT4Py backend we are using.

In [2]:
nx = 20
ny = 20
nz = 79
nhalo = 3
backend = "numpy"

## Stencil definition using GT4Py

Next, we'll define the actual stencil that we want to compile and run. We use the GT4Py domain-specific language (GTScripts) for this. For demonstration purposes, we start with a very simple stencil that simply sets a field to a constant value at every grid cell.

In [3]:
from gt4py.gtscript import PARALLEL, computation
from pace.dsl.typing import FloatField

def set_field_to_value_def(f: FloatField, value: float):
    with computation(PARALLEL), interval(...):
        f = value

## Setup helpers for building stencils

The `pace.dsl` package contains provides a helper class for compiling stencils. The helper class is called a stencil factory. In order to setup a stencil factory, several configuration objects have to be defined and passed in.

- `DaceConfig`: configuration of DaCe backend
- `CompilationConfig`: specification of how to compile
- `StencilConfig`: wrapper to carry DaCe and compilation configuration
- `GridIndexing`: helper class which provides indexing (e.g. compute domain and origin)
- `StencilFactory`: helper class to build stencils from GT4Py stencil definitions

In [4]:
from pace.dsl.dace.dace_config import DaceConfig, DaCeOrchestration
from pace.dsl.stencil import GridIndexing, StencilConfig, StencilFactory
from pace.dsl.stencil_config import CompilationConfig, RunMode

dace_config = DaceConfig(
    communicator=None, backend=backend, orchestration=DaCeOrchestration.Python
)

compilation_config = CompilationConfig(
    backend=backend,
    rebuild=True,
    validate_args=True,
    format_source=False,
    device_sync=False,
    run_mode=RunMode.BuildAndRun,
    use_minimal_caching=False,
)

stencil_config = StencilConfig(
    compare_to_numpy=False,
    compilation_config=compilation_config,
    dace_config=dace_config,
)

grid_indexing = GridIndexing(
    domain=(nx, ny, nz),
    n_halo=nhalo,
    south_edge=True,
    north_edge=True,
    west_edge=True,
    east_edge=True,
)

stencil_factory = StencilFactory(config=stencil_config, grid_indexing=grid_indexing)

## Wrap stencil into a Python class

Using the stencil factory we can now wrap the compiled GT4Py stencil into a Python class. The init section is responsible for compiling the stencil, the call section will simply execute the compiled stencil with the arguments passed.

In [12]:
from pace.dsl.stencil import StencilFactory
from pace.dsl.typing import FloatField

class SetFieldToValue:
    def __init__(self, stencil_factory: StencilFactory):
        grid_indexing = stencil_factory.grid_indexing
        self._set_field_to_value_stencil = stencil_factory.from_origin_domain(
            set_field_to_value_def,
            origin=grid_indexing.origin_compute(),
            domain=grid_indexing.domain_compute(),
        )

    def __call__(
        self,
        f: FloatField,
        value: float,
    ):
        self._set_field_to_value_stencil(f, value)

## Compiling and running the stencil

To compile and run the stencil we first have to instanciate the wrapper class. This will compile the stencil and return a callable object. Next we have to define a GT4Py data storage (field). This could also be done using the `pace.util.Quantity` class, but here we use plain GT4Py data storages for simplicity. Finally, we can call the stencil and pass in the field and value.

In [6]:
set_field_to_value = SetFieldToValue(stencil_factory)

In [7]:
import gt4py.storage as gt_storage

field = gt_storage.zeros(backend=backend, dtype=float, shape=(nx + 1*nhalo, ny + 2*nhalo, nz), default_origin=(nhalo, nhalo, 0))

In [8]:
set_field_to_value(field, 42.0)

In [9]:
print(field)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
